In [126]:
import pandas as pd
import numpy as np
import os
# import src
import matplotlib.pyplot as plt
import seaborn as sns

In [127]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from statistics import mean, stdev, median, mode
# With PCA
from sklearn.decomposition import PCA

from sklearn.feature_extraction.text import CountVectorizer
# SVM
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
# KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from mittens import GloVe, Mittens

In [128]:

import nltk
import string
from nltk.tokenize import word_tokenize
import random
import pickle
from nltk.corpus import stopwords

from autocorrect import Speller
# from pycontractions import Contractions

from spellchecker import SpellChecker

import re
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer 

# from hyperopt import fmin, tpe, hp

In [129]:
from numpy import asarray
from numpy import zeros
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [130]:
# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [131]:
def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))

    return " ".join(lemmatized_sentence)

In [132]:
data_path = '/Users/danielabraham/Downloads'
def text_preprocessing(data):
    
    # This method replaces two or more consecutive letters with the same character to something shorter. For example, gooooooood becomes good.
    def replaceTwoOrMore(s):
        #look for 2 or more repetitions of character
        pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
        return pattern.sub(r"\1\1", s)

    # This method converts camel cased words into space delimited words.
    # For example: ThisIsASentence will be changed to This Is A Sentence
    def convertCamelCase(word):
        return re.sub("([a-z])([A-Z])","\g<1> \g<2>",word)

    # Read a flat file containing some abbreviations and their expansions in pipe separated format
    # Use these abbreviations to replace text in the tweets as part of Preprocessing
    
    def readAbbrFile(abb_path):
        global abbr_dict
        abbr_dict ={}
        f = open(abb_path)
        lines = f.readlines()
        f.close()
        for i in lines:
            tmp = i.split('|')
            abbr_dict[tmp[0]] = tmp[1]

        return abbr_dict
  
    # This function checks the dictionary containing abbreviations and their meanings as (key,value) pairs
    # and replaces the key with the corresponding value
    def replaceAbbr(s):
        temp = " ".join([abbr_dict[word.lower()] if word.lower() in abbr_dict.keys() else word for word in s.split()])
        return temp
   

    def readcontractions(contra_path):
        global contra_dict
        contra_dict ={}
        f = open(contra_path)
        lines = f.readlines()
        f.close()
        for i in lines:
            try: 
                tmp = i.replace('"', '').replace(',', '').replace('\n', ' ').split(':')
                contra_dict[tmp[0]] = tmp[1]
            except:
                print(tmp)
                print(z)

        return contra_dict
    
    # This function checks the dictionary containing abbreviations and their meanings as (key,value) pairs
    # and replaces the key with the corresponding value
    def replacecontra(s):
        temp = " ".join([contra_dict[word.lower()] if word.lower() in contra_dict.keys() else word for word in s.split()])
        return temp
     
    

    abb_path = os.path.join(data_path,"abbrevations.txt")
    abbr_dict = readAbbrFile(abb_path)
    
    contra_path = os.path.join(data_path,"contractions.txt")
    contra_dict = readcontractions(contra_path)
    data["RawTweet"] = data["RawTweet"].str.replace('[^\x00-\x7f]', ' ') # remove non ascii characters
    
    data["RawTweet"] = data["RawTweet"].apply(lambda x: x.lower())
    data["RawTweet"] = data["RawTweet"].apply(lambda x: replaceAbbr(x))
    data["RawTweet"] = data["RawTweet"].apply(lambda x: replacecontra(x))

    data["RawTweet"] = data["RawTweet"].str.replace('\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*',' ') #remove URL
    data["RawTweet"] = data["RawTweet"].str.replace('(\s)@\w+', ' ') #remove usernames
    data["RawTweet"] = data["RawTweet"].str.replace('@\w+', ' ') #remove usernames
    data["RawTweet"] = data["RawTweet"].str.replace('<[^<]+?>', ' ') #remove HTML tags
    data["RawTweet"] = data["RawTweet"].str.replace('[<>!#@$:.,%\?-_]+', ' ') #remove punctuation and special characters
    
    data["RawTweet"] = data["RawTweet"].str.replace('\d+', ' ') # removing the words with more than 1 digit
    data["RawTweet"] = data["RawTweet"].str.replace('\n\n', ' ')
    data["RawTweet"] = data["RawTweet"].str.replace('\n', ' ') # removing new line characters
    data["RawTweet"] = data["RawTweet"].str.replace('[^\w\s]',' ')
    data["RawTweet"] = data["RawTweet"].str.replace('\s+',' ')  

    data["RawTweet"] = data["RawTweet"].apply(lambda x: replaceTwoOrMore(x))
    data["RawTweet"] = data["RawTweet"].apply(lambda x: convertCamelCase(x))

    # Remove stop words from text
    data["RawTweet"] = data["RawTweet"].apply(lambda x: " ".join([word for word in x.split() if word not in stop_list]))


    data["RawTweet"] = data["RawTweet"].apply(lambda x: " ".join([re.sub(r"\b(?:a{0,2}h{1,2}a{0,2}){2,}h?\b", "haha", word) for word in x.split()])) # REPLACING HAHAHAH WITH HAHA
    data["RawTweet"] = data["RawTweet"].apply(lambda x: " ".join([re.sub(r"\b(?:o{0,2}l{1,2}o{0,2}){2,}l?\b", "lol", word) for word in x.split()])) # REPLACING lOLOLOL WITH LOL
                                                  
    data["RawTweet"] = data["RawTweet"].apply(lambda x: lemmatize_sentence(x))

    return data

In [133]:
def create_vocab(data1, data2):
    temp1 = text_preprocessing(data1)
    temp2 = text_preprocessing(data2)
    return temp1, temp2

In [134]:

training_file = "/Users/danielabraham/Downloads/training-Obama-Romney-tweets1.xlsx"
obama_train1_temp = pd.read_excel(training_file,sheet_name='Obama',usecols="D,E", skiprows=[0])
romney_train1_temp = pd.read_excel(training_file,sheet_name='Romney',usecols="D,E", skiprows=[0])

obama_train1_temp.columns = ['RawTweet', 'Class']
romney_train1_temp.columns = ['RawTweet', 'Class']

## Removing datapoints with mixed sentiment

In [135]:
obama_train1 = obama_train1_temp[obama_train1_temp['Class'] .isin((1,-1,0))]
romney_train1 = romney_train1_temp[romney_train1_temp['Class'] .isin((1,-1,0))]

### Dropping Missing Values

In [136]:
obama_train1 = obama_train1.dropna()
romney_train1 = romney_train1.dropna()

## Preprocessing the data

In [137]:
stop_list = stopwords.words('english')
stop_list.extend(['rt', 'retweet', 'e'])

lemmatizer = WordNetLemmatizer() 

In [138]:
obama_train1_pr, romney_train1_pr = create_vocab(obama_train1, romney_train1)

/var/folders/rq/n9728_052hv_9kgv7hx0cf0r0000gn/T/ipykernel_56212/724192422.py:66: FutureWarning: The default value of regex will change from True to False in a future version.
  data["RawTweet"] = data["RawTweet"].str.replace('[^\x00-\x7f]', ' ') # remove non ascii characters
/var/folders/rq/n9728_052hv_9kgv7hx0cf0r0000gn/T/ipykernel_56212/724192422.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  data["RawTweet"] = data["RawTweet"].str.replace('\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*',' ') #remove URL
/var/folders/rq/n9728_052hv_9kgv7hx0cf0r0000gn/T/ipykernel_56212/724192422.py:73: FutureWarning: The default value of regex will change from True to False in a future version.
  data["RawTweet"] = data["RawTweet"].str.replace('(\s)@\w+', ' ') #remove usernames
/var/folders/rq/n9728_052hv_9kgv7hx0cf0r0000gn/T/ipykernel_56212/724192422.py:74: FutureWarning: The default value of regex will change from True to False in a future v

In [139]:
obama_train1_pr

,RawTweet,Class
0,kirkpatrick wear baseball cap embroider obama ...,0
2,obama debate cracker as cracker tonight tune t...,1
4,miss point afraid understand big picture dont ...,0
6,raise democrat leave party year ago lifetime n...,-1
7,obama camp can not afford low expectation toni...,0
...,...,...
7190,except woman work wh make less honest yup obam...,0
7192,day election selection elect lewis ken hall mh...,1
7193,reason ann romney michelle obama match last ni...,0
7194,obama kenakan cincin syahadat sejak sma,0



## Saving Processed Data

In [140]:
obama_train1_pr.to_csv(os.path.join(data_path, 'Obama_Training_Data_Preprocessed.csv'))
romney_train1_pr.to_csv(os.path.join(data_path, 'Romney_Training_Data_Preprocessed.csv'))